In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

soil = pd.read_csv("data/raw/soil_samples.csv")
soil.head()


In [ ]:
def health_score(row):
    carbon_score = (row["soil_carbon_pct"] - 2.0) / (6.0 - 2.0)
    ph_score = 1.0 - abs(row["ph"] - 6.0) / 2.0
    return max(0.0, min(1.0, 0.5 * carbon_score + 0.5 * ph_score))

soil["health_score"] = soil.apply(health_score, axis=1)
soil[["plot_id", "health_score"]].head()


In [ ]:
def label_health(score):
    if score < 0.4:
        return "low"
    elif score < 0.7:
        return "medium"
    else:
        return "high"

soil["health_class"] = soil["health_score"].apply(label_health)
soil[["plot_id", "health_score", "health_class"]].head()
soil["health_class"].value_counts()



In [ ]:
features = ["nitrogen_pct", "phosphorus_ppm", "potassium_ppm", "soil_carbon_pct", "ph"]
X = soil[features]
y = soil["health_class"]

X.head(), y.head()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_train.shape, X_test.shape


In [ ]:
clf = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt

importances = clf.feature_importances_
for name, imp in zip(features, importances):
    print(f"{name}: {imp:.3f}")

plt.figure()
plt.bar(range(len(features)), importances)
plt.xticks(range(len(features)), features, rotation=45)
plt.ylabel("Importance")
plt.title("RandomForest feature importances")
plt.tight_layout()
plt.show()
